In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from data._main import *
from train._utils import compute_loss_vectorized

import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [16]:
# When the train loop is over, load the train dataset information to retrieve the seq_len and num_train_samples
# It assumes that all worms have the same seq_len and num_train_samples (as it is in the current implementation)
log_dir = '/home/lrvnc/Projects/worm-graph/logs/hydra/2023_08_25_17_36_29'

train_dataset_info = pd.read_csv(os.path.join(log_dir, 'dataset', 'train_dataset_info.csv'))
seq_len = train_dataset_info['train_seq_len'].values[0]
num_train_samples = train_dataset_info['num_train_samples'].values[0]

# List of datasets and worms to be analyzed
analysis_config = OmegaConf.load('/home/lrvnc/Projects/worm-graph/configs/submodule/analysis.yaml').analysis
datasets_and_worms = analysis_config.validation

for dataset, worms_to_use in datasets_and_worms.items():
    
    # If no worms are specified, use all worms in the dataset...
    if isinstance(worms, list):
        # Load dataset with chosen worms
        continue

    else:
        # Load dataset with all worms
        dataset = load_dataset(dataset, seq_len=seq_len, num_train_samples=num_train_samples)
    

Kato2015
Nichols2017
Skora2018
Uzel2022
Kaplan2020
Flavell2023
Leifer2023


,Unnamed: 0,dataset,original_index,combined_dataset_index,neurons,num_neurons,train_time_steps,tau,num_train_samples,train_seq_len
0,0,Nichols2017,worm0,worm0,"['AFDR', 'ALA', 'AQR', 'ASKL', 'AUAL', 'AUAR',...",23,539,1,32,120
1,1,Nichols2017,worm1,worm1,"['AIBL', 'AIBR', 'ALA', 'AQR', 'ASKR', 'AVAL',...",36,539,1,32,120
2,2,Nichols2017,worm2,worm2,"['AFDL', 'AIBL', 'AIBR', 'ALA', 'AQR', 'AVAL',...",35,539,1,32,120
3,3,Nichols2017,worm3,worm3,"['AIBL', 'AIBR', 'ALA', 'AVAL', 'AVAR', 'AVBR'...",35,539,1,32,120
4,4,Nichols2017,worm4,worm4,"['AFDL', 'AIBL', 'AIBR', 'ALA', 'ASKL', 'ASKR'...",33,539,1,32,120
5,5,Nichols2017,worm5,worm5,"['ALA', 'ASKL', 'AVAL', 'AVAR', 'AVBL', 'AVBR'...",25,539,1,32,120
6,6,Nichols2017,worm6,worm6,"['AFDL', 'AFDR', 'AIBL', 'AIBR', 'ALA', 'AQR',...",34,539,1,32,120
7,7,Nichols2017,worm7,worm7,"['AIBL', 'AIBR', 'AQR', 'AVAL', 'AVAR', 'AVBL'...",35,539,1,32,120
8,8,Nichols2017,worm8,worm8,"['AIBL', 'AIBR', 'ALA', 'AQR', 'AUAR', 'AVAL',...",39,539,1,32,120
9,9,Nichols2017,worm9,worm9,"['AIBL', 'AIBR', 'ALA', 'AQR', 'ASKR', 'AUAR',...",30,539,1,32,120
